<h1 style="text-align:center;font-size:40px;">Tools 1 Final Project</h1>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<p style="text-align:center;">Anthony Molieri<br>
Leo Walker</p>
<br>
<br>
<br>
<br>

# Dataset and Motivation

The data used for these analyses were scraped from Indeed and Glassdoor. The data from Indeed contains job descriptions for open Data Scientist positions. The Glassdoor data includes employee generated ratings on benefits and perks for many of the companies present in the Indeed dataset. The datasets were then combined in order to give an idea of how employees rank the benefits at these companies.

*The Indeed data was scraped over ... pulling job descriptions for Data Scientists in 25 cities throughout the US. **Leo** Add Correct Details, here about the Indded data and scraper*

The Glassdoor scraper was designed to pull company names from the the job description dataset. It then looped over the list of employers, searched, and scraped ratings from the benefits page for companies that were found. This data was then added to it's own table in the same database. The scraper was designed to be run multiple times, only searching for benefits ratings that were not already present in the database. 

# Task Definition and Research Question
 
The purpose of this project was to help give a better idea of how employees at companies with open positions rate their benefits. This can help give insight into how desirable a company is as an employer.

# Literature Review

*Add -- I'm not really sure what to add here. Maybe something about turnover in data science? Companies not having great benefits?*

# Data Cleaning

In [37]:
# imports
import numpy as np
import pandas as pd
import sqlite3
import re

In [102]:
# Initialize both tables
cnct = sqlite3.connect('Data/jobs')

indeed_df = pd.read_sql_query('SELECT * FROM indeed_jobs;', cnct)
indeed_df.drop('est salary', axis=1, inplace=True)

glassdoor_df = pd.read_sql_query('SELECT companies, type, rating, count_of_ratings FROM ratings;', cnct)

In [103]:
indeed_df.columns

Index(['index', 'job_title', 'company_name', 'company_location', 'job_href',
       'job_desc', 'est_salary', 'retrieve_date'],
      dtype='object')

## Cleaning Indeed Data

In [151]:
# # Pull missing salary data from description
# salaryFix = indeed_df[['job_desc', 'est_salary']].copy()

# salaryFix['check1'] = salaryFix['job_desc'].str.contains("\$")
# salaryFix['check2'] = salaryFix['job_desc'].str.contains("salary")

# I'm not great at regex but let's try this
# Only look for salary data when it's missint in 'est_salary' 
# indeed_df['salary_extract'] = np.where(indeed_df['est_salary'] == "No Estimated Salary",indeed_df['job_desc'].str.extract("((\$)(([0-9])([0-9])([0-9])(,)([0-9])([0-9])([0-9])|([0-9])([0-9])(,)([0-9])([0-9])([0-9]))( to |-| - )(\$|)(([0-9])([0-9])([0-9])(,)([0-9])([0-9])([0-9])|([0-9])([0-9])(,)([0-9])([0-9])([0-9]))|(\$)([0-9][0-9][0-9],[0-9][0-9][0-9])|(\$)([0-9][0-9],[0-9][0-9][0-9]))")[0], np.NaN)

def salary_extract(row):
    if row['est_salary']!="No Estimated Salary":
        return row['est_salary']
    else:
        pattern = r"\$([0-9]{5,6}|[0-9]{2,3},[0-9]{3})"
        found = re.findall(pattern, row['job_desc'])
        if len(found)==1:
            return f'{found[0]} a year'
        elif len(found)>1:
            return f'{found[0]} - {found[1]} a year'
        else:
            return None
indeed_df['salary_extract'] = indeed_df.apply(salary_extract, axis=1)

In [152]:
def set_job_category(job_title):
    regex = re.compile('[^a-zA-Z]')
    clean_title = regex.sub('', job_title)
    clean_title = clean_title.lower()
    if all(x in clean_title for x in ["data", "scien"]):
        return "Data Scientist"
    elif all(x in clean_title for x in ["business", "analy"]):
        return "Business Intelligence Analyst"
    elif all(x in clean_title for x in ["bi", "analy"]):
        return "Business Intelligence Analyst"
    elif all(x in clean_title for x in ["data", "analy"]):
        return "Data Analyst"
    elif all(x in clean_title for x in ["machine", "learning"]):
        return "Machine Learning Engineer"
    elif all(x in clean_title for x in ["data", "engineer"]):
        return "Data Engineer"
    elif all(x in clean_title for x in ["software", "engin"]):
        return "Software Engineer"
    elif all(x in clean_title for x in ["architect"]):
        return "Data Architect"
    else:
        return "Other"
indeed_df['job_category'] = indeed_df.loc[:, 'job_title'].apply(set_job_category)

In [153]:
indeed_df.loc[:,'state'] = indeed_df.loc[:, 'company_location'].str.extract(r', ([A-Z]{2})')

In [154]:
# indeed_df[indeed_df['salary_extract']!=None]
for i in indeed_df.salary_extract:
    print(i)

None
None
None
None
None
None
$56,000 - $106,080 a year
None
None
None
$72,500 - $137,400 a year
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
$100,800 - $230,200 a year
None
None
None
None
None
$110,500 a year
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
$60,000 - $90,000 a year
$105,400 a year
$123,300 - $160,000 a year
78,718 - 129,885 a year
84,300 - 125,999 a year
None
90,000 - 150,000 a year
None
$100,800 - $230,200 a year
None
86,200 - 176,600 a year
$120,700 a year
$120,700 - $160,000 a year
90,000 - 140,000 a year
$145,200 - $217,800 a year
None
None
None
None
None
None
None
None
None
None
64,800 - 116,000 a year
None
None
None
None
None
None
None
None
84,300 - 125,999 a year
None
None
None
None
None
None
None
None
None
None
None
None
$75,504 - $94,761 a year
None
None
None
None
None
None
None
None
None
None
None
None
None
None
20,000 a year
None
None
None
Non

In [155]:
# indeed_df.info()
# indeed_df.loc[indeed_df['est_salary']=="No Estimated Salary"]
# indeed_df.loc[236,'job_desc']

In [156]:
def salary_low(row):
    if row =='No Estimated Salary':
        return None
    salary = str(row).split('-')
    pattern = r'([0-9]*)'
    if len(salary)>1:
        low = int(''.join(re.findall(pattern, salary[0])))
    else:
        low = int(''.join(re.findall(pattern, salary[0])))
    return low

def salary_high(row):
    if row =='No Estimated Salary':
        return None
    salary = str(row).split('-')
    pattern = r'([0-9]*)'
    if len(salary)>1:
        high = int(''.join(re.findall(pattern, salary[1])))
    else:
        low = high = int(''.join(re.findall(pattern, salary[0])))
    return high

indeed_df.loc[:,['salary_low']] = indeed_df.loc[:,'est_salary'].apply(salary_low)
indeed_df.loc[:,['salary_high']] = indeed_df.loc[:,'est_salary'].apply(salary_high)

In [157]:
def pay_rate(row):
    regex = re.compile('[.[0-9]]')
    clean_salary = regex.sub('', row)
    clean_salary = clean_salary.lower()
    if any(x in clean_salary for x in ["year"]):
        return "annual"
    elif any(x in clean_salary for x in ["month"]):
        return "month"
    elif any(x in clean_salary for x in ["week"]):
        return "week"
    elif any(x in clean_salary for x in ["day"]):
        return "day"
    elif any(x in clean_salary for x in ["hour"]):
        return "hour"
    else:
        return "Other"

indeed_df['pay_rate'] = indeed_df.loc[:, 'est_salary'].apply(pay_rate)

In [159]:
def calc_salary(row):
    # print(row)
    pay_rate=row['pay_rate']
    if pay_rate == 'Other':
        return None

    rate_change= {'annual':1, 'month':12, 'week':52, 'day':260, 'hour':2080}
    est_annual_salary = ((row['salary_low']+row['salary_high'])/2)*rate_change[pay_rate]
    return est_annual_salary
indeed_df['est_annual_salary'] = indeed_df.loc[:,['est_salary', 'salary_low','salary_high','pay_rate']].apply(calc_salary, axis=1)


In [160]:
indeed_df[indeed_df['est_salary']!='No Estimated Salary'].sample(4)

,index,job_title,company_name,company_location,job_href,job_desc,est_salary,retrieve_date,job_category,state,salary_extract,salary_low,salary_high,pay_rate,est_annual_salary
16346,11,Business Systems Analyst,MADD,"Hybrid remote in Irving, TX 75062",https://www.indeed.com/company/MADD/jobs/Busin...,GENERAL DESCRIPTION: Business Systems Analys...,"$80,000 - $100,000 a year",2022-04-11,Business Intelligence Analyst,NaN,"$80,000 - $100,000 a year",80000.0,100000.0,annual,90000.0
9224,7,Business Analyst - Strategic Product Growth & ...,Wilbur-Ellis Company,"Remote in Denver, CO 80216",https://www.indeed.com/rc/clk?jk=05a1a871d028a...,Wilbur-Ellis is one of the largest family-owne...,"$85,000 - $110,000 a year",2022-03-14,Business Intelligence Analyst,NaN,"$85,000 - $110,000 a year",85000.0,110000.0,annual,97500.0
13731,4,Sr. Data Analyst,FutureRecruit,"Remote in Lehi, UT 84043",https://www.indeed.com/rc/clk?jk=a6cd25ea3e02f...,Sr. Data Analyst (100% REMOTE) Position Ty...,$42 - $45 an hour,2022-04-04,Data Analyst,NaN,$42 - $45 an hour,42.0,45.0,hour,90480.0
13307,1,Business Data Analyst,Cascade Public Media,"Seattle, WA 98109 (Uptown area)",https://www.indeed.com/rc/clk?jk=bf390f9bfb659...,"As the Business Data Analyst, you will creat...","$70,500 - $80,000 a year",2022-04-04,Business Intelligence Analyst,NaN,"$70,500 - $80,000 a year",70500.0,80000.0,annual,75250.0


# Visualizations